เราสามารถประยุกต์ใช้ Callback ทำงานได้หลากหลาย เนื่องจากเราจะรู้ถึง Event ต่าง ๆ ที่เกิดขึ้นภายใน Training Loop ทุกขั้นตอน และ สามารถเข้าถึงข้อมูล Model, Runner, DataBunch, Loss Function, etc. ได้ทั้งหมด 

เรามาดู[ตัวอย่างการเขียน Callback ทำงานต่าง ๆ เช่น คำนวน Metrics, ย้ายข้อมูลขึ้น CUDA, Learning Rate Annealing, etc. เริ่มที่หัวข้อ 6](#6.-Callbacks)

แต่ Callback ก็มีข้อจำกัดอยู่ คือ เราไม่สามารถเข้าถึง Layer ต่าง ๆ หรือ Event ภายในโมเดลได้ เนื่องจากโมเดลที่เราใช้เป็นการเรียกใช้ API ของ PyTorch อีกต่อหนึ่ง ในกรณีที่เราต้องการเข้าถึงภายในโมเดล PyTorch ก็มี Callback ให้เราใช้เหมือนกัน เรียกว่า Hook จะอธิบายต่อไป

# 0. Magic

In [0]:
%load_ext autoreload
%autoreload 2
%matplotlib inline

# 1. Import

In [0]:
import torch
from torch import tensor
from torch.nn import *
import torch.nn.functional as F
from torch.utils.data import *
from fastai import datasets
from fastai.metrics import accuracy
import pickle, gzip, math, torch, re
from IPython.core.debugger import set_trace

# 2. Data

In [0]:
class Dataset(Dataset):
    def __init__(self, x, y):
        self.x, self.y = x, y
    def __len__(self):
        return len(self.x)
    def __getitem__(self, i):
        return self.x[i], self.y[i]

In [0]:
MNIST_URL='http://deeplearning.net/data/mnist/mnist.pkl'

In [0]:
def get_data():
    path = datasets.download_data(MNIST_URL, ext='.gz')
    with gzip.open(path, 'rb') as f:
        ((x_train, y_train), (x_valid, y_valid), _) = pickle.load(f, encoding='latin-1')
    return map(tensor, (x_train, y_train, x_valid, y_valid))

In [0]:
x_train, y_train, x_valid, y_valid = get_data()

In [0]:
def normalize(x, m, s): 
    return (x-m)/s

In [0]:
from typing import *

def listify(o):
    if o is None: return []
    if isinstance(o, list): return o
    if isinstance(o, str): return [o]
    if isinstance(o, Iterable): return list(o)
    return [o]

In [0]:
_camel_re1 = re.compile('(.)([A-Z][a-z]+)')
_camel_re2 = re.compile('([a-z0-9])([A-Z])')

def camel2snake(name):
    s1 = re.sub(_camel_re1, r'\1_\2', name)
    return re.sub(_camel_re2, r'\1_\2', s1).lower()   

In [0]:
train_mean, train_std = x_train.mean(), x_train.std()
x_train = normalize(x_train, train_mean, train_std)
x_valid = normalize(x_valid, train_mean, train_std)

In [0]:
nh, bs = 100, 32
n, m = x_train.shape
c = (y_train.max()+1).numpy()
loss_func = F.cross_entropy

In [0]:
train_ds, valid_ds = Dataset(x_train, y_train), Dataset(x_valid, y_valid)
train_dl, valid_dl = DataLoader(train_ds, bs), DataLoader(valid_ds, bs)

# 3. DataBunch

In [0]:
class DataBunch():
    def __init__(self, train_dl, valid_dl, c=None):
        self.train_dl,self.valid_dl,self.c = train_dl,valid_dl,c

    @property
    def train_ds(self): return self.train_dl.dataset

    @property
    def valid_ds(self): return self.valid_dl.dataset

In [0]:
data = DataBunch(train_dl, valid_dl, c)

# 4. Model

In [0]:
lr = 0.03
epoch = 10
nh = 50

In [0]:
def get_model():
    # loss function
    loss_func = F.cross_entropy
    model = Sequential(Linear(m, nh), ReLU(), Linear(nh,c))
    return model, loss_func

In [0]:
model, loss_func = get_model()
opt = torch.optim.SGD(model.parameters(), lr=lr)

In [0]:
class Learner():
    def __init__(self, model, opt, loss_func, data):
        self.model, self.opt, self.loss_func, self.data = model, opt, loss_func, data


In [0]:
learn = Learner(model, opt, loss_func, data)

# 5. Training Loop

Training Loop ที่รองรับ Callback

In [0]:
class Runner():
    def __init__(self, cbs=None, cb_funcs=None):
        cbs = listify(cbs)
        for cbf in listify(cb_funcs):
            cb = cbf()
            setattr(self, cb.name, cb)
            cbs.append(cb)
        self.stop, self.cbs = False, [TrainEvalCallback()]+cbs

    @property
    def opt(self):          return self.learn.opt
    @property
    def model(self):        return self.learn.model
    @property
    def loss_func(self):    return self.learn.loss_func
    @property
    def data(self):         return self.learn.data

    def one_batch(self, xb, yb):
        try: 
            self.xb, self.yb = xb, yb
            self('begin_batch')
            self.pred = self.model(xb)
            self('after_pred')
            self.loss = self.loss_func(self.pred, yb)
            self('after_loss')
            if not self.in_train: return
            self.loss.backward()
            self('after_backward')
            self.opt.step()
            self('after_step')
            self.opt.zero_grad()
        except CancelBatchException: self('after_cancel_batch')
        finally: self('after_batch')
    
    def all_batches(self, dl):
        self.iters = len(dl)
        try:
            for xb, yb in dl:
                self.one_batch(xb, yb)
        except CancelEpochException: self('after_cancel_epoch')
    
    def fit(self, epochs, learn):
        self.epochs, self.learn, self.loss = epochs, learn, tensor(0.)

        try:
            for cb in self.cbs: cb.set_runner(self)
            self('begin_fit')
            for epoch in range(epochs):
                self.epoch = epoch
                if not self('begin_epoch'): self.all_batches(self.data.train_dl)

                with torch.no_grad():
                    if not self('begin_validate'): self.all_batches(self.data.valid_dl)
                self('after_epoch')
        except CancelTrainException: self('after_cancel_train')
        finally: 
            self('after_fit')
            self.train = None

    def __call__(self, cb_name):
        # return True = Cancel, return False = Continue (Default)
        res = False
        # check if at least one True return True
        for cb in sorted(self.cbs, key=lambda x: x._order): res = res or cb(cb_name)
        return res        

In [0]:
class Callback():
    _order = 0
    def set_runner(self, run): self.run = run
    def __getattr__(self, k): return getattr(self.run, k)

    @property
    def name(self):
        name = re.sub(r'Callback$', '', self.__class__.__name__)
        return camel2snake(name or 'callback')
    
    def __call__(self, cb_name):
        f = getattr(self, cb_name, None)
        if f and f(): return True
        return False

class TrainEvalCallback(Callback):
    def begin_fit(self):
        self.run.n_epochs = 0.
        self.run.n_iter = 0
    
    def begin_epoch(self):
        self.run.n_epochs = self.epoch  
        self.model.train()
        self.run.in_train=True

    def after_batch(self):
        if not self.in_train: return
        self.run.n_epochs += 1./self.iters
        self.run.n_iter += 1

    def begin_validate(self):
        self.model.eval()
        self.run.in_train=False    
           
class CancelTrainException(Exception): pass
class CancelEpochException(Exception): pass
class CancelBatchException(Exception): pass

# 6. Callbacks

เริ่มกันที่ Callback คำนวน Metrics

In [0]:
class AvgStats():
    def __init__(self, metrics, in_train): 
        self.metrics, self.in_train = listify(metrics), in_train
    
    def reset(self):
        self.tot_loss, self.count = 0., 0
        self.tot_mets = [0.] * len(self.metrics)

    @property
    def all_stats(self): return [self.tot_loss.item()] + self.tot_mets
    @property
    def avg_stats(self): return [o/self.count for o in self.all_stats]

    def __repr__(self):
        if not self.count: return ""
        return f"{'train' if self.in_train else 'valid'}: {self.avg_stats}"

    def accumulate(self, run):
        bn = run.xb.shape[0]
        self.tot_loss += run.loss * bn
        self.count += bn
        
        for i, m in enumerate(self.metrics):
            self.tot_mets[i] += m(run.pred, run.yb) * bn


In [0]:
class AvgStatsCallback(Callback):
    def __init__(self, metrics):
        self.train_stats, self.valid_stats = AvgStats(metrics, True), AvgStats(metrics, False)
    def begin_epoch(self):
        self.train_stats.reset()
        self.valid_stats.reset()
    
    def after_loss(self):
        stats = self.train_stats if self.in_train else self.valid_stats
        with torch.no_grad(): stats.accumulate(self.run)

    def after_epoch(self):
        print(self.train_stats)
        print(self.valid_stats)

In [0]:
learn = Learner(model, opt, loss_func, data)

In [0]:
stats = AvgStatsCallback([accuracy])
run = Runner(cbs=stats)

In [28]:
run.fit(5, learn)

train: [0.352501796875, tensor(0.9003)]
valid: [0.21447978515625, tensor(0.9402)]
train: [0.186612890625, tensor(0.9456)]
valid: [0.1538888671875, tensor(0.9583)]
train: [0.138806337890625, tensor(0.9599)]
valid: [0.12684234619140625, tensor(0.9638)]
train: [0.11247166015625, tensor(0.9676)]
valid: [0.1141068115234375, tensor(0.9669)]
train: [0.094946962890625, tensor(0.9728)]
valid: [0.106433935546875, tensor(0.9683)]


In [32]:
loss, acc = stats.valid_stats.avg_stats
loss, acc

(0.106433935546875, tensor(0.9683))

In [0]:
from functools import partial

In [0]:
acc_cbf = partial(AvgStatsCallback, accuracy)

In [36]:
run = Runner(cb_funcs=acc_cbf)
run.fit(5, learn)

train: [0.081978310546875, tensor(0.9765)]
valid: [0.10045712890625, tensor(0.9707)]
train: [0.071851044921875, tensor(0.9798)]
valid: [0.09716101684570312, tensor(0.9719)]
train: [0.063715380859375, tensor(0.9822)]
valid: [0.094887353515625, tensor(0.9716)]
train: [0.056918984375, tensor(0.9839)]
valid: [0.0929585693359375, tensor(0.9725)]
train: [0.0510165771484375, tensor(0.9861)]
valid: [0.093050927734375, tensor(0.9742)]


In [37]:
run.avg_stats.valid_stats.avg_stats

[0.093050927734375, tensor(0.9742)]